In [2]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine
import geopandas as gpd
from shapely.geometry import box
from rasterio.warp import calculate_default_transform, reproject

## Check a raster coordinates reference system and projected coordinates system


## check aviris brdf reference system

In [2]:


# Open the image file
with rasterio.open(r"F:\wenqu\Aviris\2022\output\ang20220706t194511_rfl_v2aa2_img_brdf") as dataset:
    # Print the coordinate reference system (CRS)
    print("Coordinate Reference System (CRS):", dataset.crs)
    print(f"Extent: {dataset.bounds}")
    
    # Print the geographic coordinate system (GCS)
    if dataset.crs.is_geographic:
        print("Geographic Coordinate System (GCS):", dataset.crs)
    else:
        print("The CRS is not a geographic coordinate system.")
        
    # Print the projected coordinate system (PCS)
    if dataset.crs.is_projected:
        print("Projected Coordinate System (PCS):", dataset.crs)
    else:
        print("The CRS is not a projected coordinate system.")
        
        
        
        
        
        

Coordinate Reference System (CRS): EPSG:32605
Extent: BoundingBox(left=604534.1835526649, bottom=7418595.916271065, right=630957.4580532165, top=7491930.7715)
The CRS is not a geographic coordinate system.
Projected Coordinate System (PCS): EPSG:32605


## check trait img reference system

In [3]:
# Open the image file
with rasterio.open(r'F:\wenqu\trait_map\site2a_la_trait_map.tiff') as dataset:
    # Print the coordinate reference system (CRS)
    print("Coordinate Reference System (CRS):", dataset.crs)
    print(f"Extent: {dataset.bounds}")
    
    # Print the geographic coordinate system (GCS)
    if dataset.crs.is_geographic:
        print("Geographic Coordinate System (GCS):", dataset.crs)
    else:
        print("The CRS is not a geographic coordinate system.")
        
    # Print the projected coordinate system (PCS)
    if dataset.crs.is_projected:
        print("Projected Coordinate System (PCS):", dataset.crs)
    else:
        print("The CRS is not a projected coordinate system.")

Coordinate Reference System (CRS): EPSG:4326
Extent: BoundingBox(left=-150.105644848222, bottom=67.38120682858906, right=-150.1019170547879, top=67.3825636464704)
Geographic Coordinate System (GCS): EPSG:4326
The CRS is not a projected coordinate system.


## project raster 2 using raster 1

In [3]:
# Open raster1 and raster2
with rasterio.open(r"F:\wenqu\Aviris\2022\output\ang20220706t194511_rfl_v2aa2_img_brdf") as src1:
    with rasterio.open(r'F:\wenqu\trait_map\site1b_d15n_trait.tif') as src2:
        
        # Get the CRS and bounds of raster1
        dst_crs = src1.crs
        dst_bounds = src1.bounds
        
        # Calculate the transform for raster2 to match raster1
        transform, width, height = calculate_default_transform(
            src2.crs, dst_crs, src2.width, src2.height, *src2.bounds)
        
        # Update the metadata of raster2 to match the new CRS and bounds
        dst_profile = src2.profile.copy()
        dst_profile.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'bounds': dst_bounds
        })
        
        # Reproject raster2 to match raster1
        with rasterio.open(r"F:\wenqu\Aviris\2022\trait_map\trait_map_projection\site1b_d15n_trait_proj.tif", 'w', **dst_profile) as dst:
            for i in range(1, src2.count + 1):
                reproject(
                    source=rasterio.band(src2, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src2.transform,
                    src_crs=src2.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=rasterio.warp.Resampling.bilinear
                )
